In [1]:
#importy
import sys
import io

import static_ev as ev
from data_import import load_onedrive_csv
from tokenizacja import tokenize_column

In [2]:
# pobranie danych z onedrive
indeksy_df = load_onedrive_csv(ev.indeksy_url)
kombinacje_df = load_onedrive_csv(ev.kombinacje_url)

Pobieranie danych z OneDrive...
Pomyślnie zdekodowano przy użyciu utf-8
Pomyślnie zdekodowano przy użyciu utf-8


c:\Users\rafalkaminski\OneDrive - Gdańska Stocznia „Remontowa” im. J.Piłsudskiego S.A\Documents\GSR\studia\ML_PROJEKT\data_import.py:40: DtypeWarning: Columns (30,34,53,57,75,93,142,143,144,157,159,166,174,176,183,239,240,252,256,265) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(StringIO(csv_content), sep=';')


Wczytano 108782 wierszy i 266 kolumn
Pobieranie danych z OneDrive...
Pomyślnie zdekodowano przy użyciu windows-1250
Pomyślnie zdekodowano przy użyciu windows-1250
Wczytano 386148 wierszy i 13 kolumn
Wczytano 386148 wierszy i 13 kolumn


In [3]:
#czyszczenie danych
indeksy_df = indeksy_df[['ID', 'INDEKS', 'NAZWA','KOMB_ID']]
kombinacje_df = kombinacje_df[['SEGM_ID','KOMB_ID', 'POZYCJA_SEG','SLIT_ID','SLOW_ID', 'KOD', 'OPIS', 'ZALEZNY_OD_ID','NAZWA_SLOW', 'ZALEZNY_OD_SLOW_ID']]

indeksy_df= indeksy_df.dropna(subset=['KOMB_ID'])

In [4]:
#tokenizacja kolumny nazwa w ramce indeksy_df
indeksy_df_token = tokenize_column(indeksy_df, column_name='NAZWA', model='gpt-3.5-turbo')

Tokenizacja zakończona. Średnia liczba tokenów: 26.43
Maksymalna liczba tokenów: 116
Minimalna liczba tokenów: 1


In [5]:
indeksy_df_token.head()

,ID,INDEKS,NAZWA,KOMB_ID,NAZWA_tokens,NAZWA_token_count
173,192,"M-RUR-KOL-76,1-S235-6-46055","KOŁNIERZ Z SZYJKĄ TYP 11 B DN65 76,1X2,9 PN10/...",21.0,"[56047, 129, 223, 15259, 643, 57, 1901, 70344,...",39
869,888,"M-RUR-ZSC-139,7-X52-6-65355","RURA 139,7X6,3 X52 PSL1 AT 3.1",22.0,"[49, 41410, 4194, 10125, 11, 22, 55, 21, 11, 1...",21
870,889,"M-RUR-ZSC-168,3-X52-6-65353","RURA 168,3X6,3 X52 PSL1 AT 3.1",23.0,"[49, 41410, 4194, 8953, 11, 18, 55, 21, 11, 18...",21
872,891,"M-RUR-ZSC-168,3-X52-6-65354","RURA 168,3X8 X52 PSL1 AT 3.1",24.0,"[49, 41410, 4194, 8953, 11, 18, 55, 23, 4194, ...",20
879,898,U-POM-AMP-EA19-0-0-463,AMPEROMIERZ TABLICOWY EA19 CT=400/5A,25.0,"[1428, 9851, 1937, 16688, 57, 50276, 43, 1341,...",18


In [6]:
# Ogranicz dane z indeksy_df_token do takich wierszy, które w kolumnie Indeks zaczynają się od 'M' 
indeksy_df_token = indeksy_df_token[indeksy_df_token['INDEKS'].str.startswith('M')]
# wyświetlenie 10 rekordów z najmniejszą liczbą tokenów w kolumnie 'token_count'
print(indeksy_df_token.nsmallest(10, 'NAZWA_token_count'))
# Zaznacz kolorem tła fragment teksu w kolumnie nazwa odpowiadajacy poscególnemu tokenowi w kolumnie 'NAZWA_tokens' i wyświetl wynik
 

          ID                   INDEKS                NAZWA  KOMB_ID  \
28793  28813    M-CHE-NAF-0-0-0-50812                NAFTA  28274.0   
45096  45118  M-OPA-CEL-KAR-0-0-58873               KARTON  43502.0   
8463    8482    M-LLZ-RLA-A-SS-0-7183           ROLKA A SS   7717.0   
37945  37966   M-LLZ-RLA-A-FE-0-48902           ROLKA A FE  37316.0   
21256  21276    M-CHE-OLE-0-0-0-18720           OLEJ SAE40  20849.0   
21257  21277    M-CHE-OLE-0-0-0-18721           OLEJ SAE50  20850.0   
26028  26048   M-ETR-TRA-YT-0-6-36270  TRANSFORMATOR YT105  25590.0   
26029  26049   M-ETR-TRA-YT-0-6-36268   TRANSFORMATOR YT50  25591.0   
26030  26050   M-ETR-TRA-YT-0-6-36269   TRANSFORMATOR YT70  25592.0   
26082  26102   M-ETR-TRA-YT-0-6-36271  TRANSFORMATOR YT150  25642.0   

                             NAZWA_tokens  NAZWA_token_count  
28793                       [7476, 59512]                  2  
45096                     [42, 3065, 715]                  3  
8463           [16922, 27542,

In [7]:
 # Na podstawie indeksy_df_token stóworz ramkę danych z kolumnami: INDEKS, NAZWA, NAZWA_tokens, NAZWA_token_count, Indeks_2_bloki  w której Indeks_2_bloki to wartość z kolumny INDEKS po drógim znaku '-'
indeksy_df_token['Indeks_2_bloki'] = indeksy_df_token['INDEKS'].apply(lambda x: '-'.join(x.split('-')[:2]) if '-' in x else x)
indeksy_model_1 = indeksy_df_token[['INDEKS', 'NAZWA', 'NAZWA_tokens', 'NAZWA_token_count', 'Indeks_2_bloki']]  
indeksy_model_1.head()  



,INDEKS,NAZWA,NAZWA_tokens,NAZWA_token_count,Indeks_2_bloki
173,"M-RUR-KOL-76,1-S235-6-46055","KOŁNIERZ Z SZYJKĄ TYP 11 B DN65 76,1X2,9 PN10/...","[56047, 129, 223, 15259, 643, 57, 1901, 70344,...",39,M-RUR
869,"M-RUR-ZSC-139,7-X52-6-65355","RURA 139,7X6,3 X52 PSL1 AT 3.1","[49, 41410, 4194, 10125, 11, 22, 55, 21, 11, 1...",21,M-RUR
870,"M-RUR-ZSC-168,3-X52-6-65353","RURA 168,3X6,3 X52 PSL1 AT 3.1","[49, 41410, 4194, 8953, 11, 18, 55, 21, 11, 18...",21,M-RUR
872,"M-RUR-ZSC-168,3-X52-6-65354","RURA 168,3X8 X52 PSL1 AT 3.1","[49, 41410, 4194, 8953, 11, 18, 55, 23, 4194, ...",20,M-RUR
880,M-SRU-POD-M20-0-0-468,PODKŁADKA ZGRUBNA 22 FEZN OGNIOWO POKRYCIE FLU...,"[2089, 18805, 129, 223, 1846, 27542, 1901, 879...",29,M-SRU


In [9]:
import sys
print(sys.executable)
import sklearn
print(sklearn.__version__)


c:\Users\rafalkaminski\OneDrive - Gdańska Stocznia „Remontowa” im. J.Piłsudskiego S.A\Documents\GSR\studia\ML_PROJEKT\.venv\Scripts\python.exe
1.7.2


In [13]:
# Korzystajac z XGBRegressor z biblioteki xgboost stwórz model regresji przewidujący wartość z kolumny 'Indeks_2_bloki'  na wartosci liczbowych zapisanych w kolenie 'Nazwa_tokens', każda wartość po przecinku powinna byc osobnym wektorem cech. Podziel dane na zbiór treningowy i testowy w stosunku 80/20. Wyświetl raport z wynikami modelu na zbiorze testowym.  
#from xgboost import XGBRegressor 
from xgboost import XGBClassifier
from sklearn.model_selection import train_test_split    
from sklearn.metrics import mean_squared_error, r2_score
import numpy as np

max_tokens = indeksy_model_1['NAZWA_tokens'].apply(len).max()
X = np.array([
    np.pad(tokens, (0, max_tokens - len(tokens)), constant_values=0)[:max_tokens]
    for tokens in indeksy_model_1['NAZWA_tokens']
])
y = indeksy_model_1['Indeks_2_bloki'].astype('category').cat.codes  
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
#model = XGBRegressor(objective='reg:squarederror', n_estimators=100, learning_rate=0.1, max_depth=5, random_state=42)
model = XGBClassifier(n_estimators=100, learning_rate=0.1, max_depth=5)
model.fit(X_train, y_train)
y_pred = model.predict(X_test)
mse = mean_squared_error(y_test, y_pred)    
r2 = r2_score(y_test, y_pred)
print(f'Mean Squared Error: {mse}')
print(f'R^2 Score: {r2}')


Mean Squared Error: 63.29306104196169
R^2 Score: 0.8630017666835526


In [ ]:
# Użyj tokenów jako cech zamiast tylko ich liczby
from sklearn.preprocessing import LabelEncoder
import numpy as np

# Przekształć tokeny na macierz cech (np. średnia tokenów, TF-IDF, embeddingi)
max_tokens = indeksy_model_1['NAZWA_tokens'].apply(len).max()
X = np.array([
    np.pad(tokens, (0, max_tokens - len(tokens)), constant_values=0)[:max_tokens]
    for tokens in indeksy_model_1['NAZWA_tokens']
])

# Lub użyj XGBClassifier zamiast XGBRegressor
from xgboost import XGBClassifier
model = XGBClassifier(n_estimators=100, learning_rate=0.1, max_depth=5)

In [12]:
from tokenizacja import decode_tokens

# dekodowanie przykładowej wartości tokenów z ramki indeksy_df_token
sample_tokens = indeksy_df_token.iloc[3]['NAZWA_tokens']
decoded_text = decode_tokens(sample_tokens)
print(f"Oryginalna nazwa: {indeksy_df_token.iloc[3]['NAZWA']}")
print(f"Zdekodowany tekst: {decoded_text}")
print(f"Liczba tokenów: {len(sample_tokens)}")

Oryginalna nazwa: RURA 168,3X8 X52 PSL1 AT 3.1
Zdekodowany tekst: RURA 168,3X8 X52 PSL1 AT 3.1
Liczba tokenów: 20


### Kernel i weryfikacja środowiska
Upewnij się, że notebook używa kernela `ML_PROJEKT (.venv)`.
Jeśli nie, wybierz go w VS Code: Command Palette → Notebook: Select Notebook Kernel → wybierz 'ML_PROJEKT (.venv)'.
Po wybraniu kernela uruchom komórkę testową poniżej (kod Python).

In [ ]:
# Sprawdzenie interpretera i importów
import sys
print('sys.executable:', sys.executable)

# xgboost
try:
    import xgboost
    print('xgboost version:', xgboost.__version__)
except Exception as e:
    print('xgboost import error:', type(e).__name__, e)

# scikit-learn
try:
    import sklearn
    print('scikit-learn version:', sklearn.__version__)
except Exception as e:
    print('sklearn import error:', type(e).__name__, e)